* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

# 1) 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 두 데이타셋을 보기 편하게 합쳤다.
data = pd.merge(ratings, movies, how='left', on='movie_id')
data.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [7]:
# 영화 수
data['movie_id'].nunique()

3628

In [8]:
# 유저 수
data['user_id'].nunique()

6039

In [9]:
# 인기 많은 영화
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [10]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

In [11]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [12]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Matrix, The (1999)' , 'Forrest Gump (1994)' ,'Terminator, The (1984)' ,'Toy Story (1995)' ,'Men in Black (1997)']
movie_id_f = [2571,356,1240,1,1580]
# 'zimin'이라는 user_id가 위 영화의 노래를 30회씩 들었다고 가정하겠습니다.
my_title_list = pd.DataFrame({'user_id': [7000]*5,'movie_id':movie_id_f,'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':[7000]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_title_list)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 


data = data.drop(columns={'timestamp','genre'})
data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,7000,2571,5,"Matrix, The (1999)"
1,7000,356,5,Forrest Gump (1994)
2,7000,1240,5,"Terminator, The (1984)"
3,7000,1,5,Toy Story (1995)
4,7000,1580,5,Men in Black (1997)


In [13]:
# (참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.

# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[7000])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['Matrix, The (1999)'])

6039
124


In [20]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data.head(50)

user_id column indexing Fail!!
movie column indexing Fail!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
5,0,1197,3,5
6,0,1287,5,6
7,0,2804,5,7
8,0,594,4,8
9,0,919,4,9


# 4) CSR matrix를 직접 만들어 봅시다.

In [21]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data


<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose
# 모델 훈련
als_model.fit(csr_data_transpose)


  0%|          | 0/15 [00:00<?, ?it/s]

In [26]:
jenny, Forrest_Gump = user_to_idx[7000], title_to_idx['Forrest Gump (1994)'] 
jenny_vector, Forrest_Gump_vector = als_model.user_factors[jenny], als_model.item_factors[Forrest_Gump]
print(jenny_vector, Forrest_Gump_vector)
print('슝=3')

[-0.1717587  -0.2618403   0.5333038   0.0988498   0.5753729   0.02347128
 -0.4204156   0.81159973  0.10458658  0.11098973  0.6253729  -0.75977105
  0.2728019   0.11515069 -1.0944217  -0.48429155  0.11065701  0.24924739
 -0.3810153   1.050057   -0.359628    0.5953815   0.2128866   0.11506132
  0.5145879  -0.467406    0.27552268  0.1390658  -0.2968903  -0.9570177
 -0.19209865 -0.5041043  -0.49761373 -0.0919872   0.67301226  0.2815861
 -0.54617625  0.257681    0.28628123 -0.07958183  0.9222344   0.00997066
  0.21484868 -0.21217261  0.11845401  0.59244865 -0.21650797  1.0354348
  0.43976477  0.05023014 -0.5744729   0.309175    0.11286704 -0.22338197
  0.282824    0.90625805 -0.16096942 -0.56669384 -0.4161699   0.31177694
  0.40378138 -0.09101252  0.09733599  0.05793758 -0.43707705 -0.3834937
 -0.632338   -0.05405891  0.6397972  -0.6321398  -1.048109   -0.30392024
  0.40190187  1.0254247  -0.22756356  0.6321889  -0.64357394  0.42211422
  0.12378703  0.07407963  0.34093457 -0.11761637 -0.016

In [27]:
# jenny,Forrest_Gump  내적하는 코드
np.dot(jenny_vector, Forrest_Gump_vector)

0.59282494

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


In [28]:
# 유저 선호도
favorite_movie = 'Matrix, The (1999)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 1.0),
 (92, 0.7835702),
 (62, 0.67463833),
 (141, 0.5919898),
 (200, 0.5774761),
 (107, 0.54731077),
 (145, 0.5453823),
 (375, 0.51548326),
 (317, 0.49847624),
 (175, 0.49573335),
 (75, 0.4931727),
 (44, 0.44066378),
 (236, 0.41824707),
 (193, 0.41610315),
 (117, 0.41305232)]

In [30]:
#title_to_idx 를 뒤집어, index로부터 title 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Jurassic Park (1993)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Twelve Monkeys (1995)',
 'Men in Black (1997)',
 'Hunt for Red October, The (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Speed (1994)',
 'Alien (1979)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)']

In [33]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [35]:
get_similar_movie('Jurassic Park (1993)')

['Jurassic Park (1993)',
 'Men in Black (1997)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Lost World: Jurassic Park, The (1997)',
 'Braveheart (1995)',
 'Mission: Impossible (1996)',
 'Face/Off (1997)']

In [37]:
# 유사도
user = user_to_idx[7000]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.64362353),
 (107, 0.5227454),
 (64, 0.4675493),
 (50, 0.44932312),
 (62, 0.41131002),
 (110, 0.3548724),
 (117, 0.35095102),
 (22, 0.3016684),
 (651, 0.28987598),
 (141, 0.28819478),
 (44, 0.28587896),
 (4, 0.28257865),
 (38, 0.2551699),
 (60, 0.25226444),
 (322, 0.24068013),
 (150, 0.22754145),
 (121, 0.22742157),
 (193, 0.21835545),
 (33, 0.21828964),
 (865, 0.2136462)]

In [39]:
[idx_to_title[i[0]] for i in movie_recommended]

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Toy Story 2 (1999)',
 'Total Recall (1990)',
 'Groundhog Day (1993)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Back to the Future (1985)',
 'Aliens (1986)',
 'Fugitive, The (1993)',
 'Star Wars: Episode IV - A New Hope (1977)',
 "Bug's Life, A (1998)",
 'Sixth Sense, The (1999)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Babe (1995)',
 'Independence Day (ID4) (1996)',
 'Silence of the Lambs, The (1991)',
 'Alien (1979)',
 'Aladdin (1992)',
 'Predator (1987)']

In [40]:
back_to_the_future = title_to_idx['Back to the Future (1985)']
explain = als_model.explain(user, csr_data, itemid=back_to_the_future)

In [41]:
# 기여도
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Forrest Gump (1994)', 0.09802175308870056),
 ('Terminator, The (1984)', 0.0834288089915445),
 ('Matrix, The (1999)', 0.05838803441207332),
 ('Toy Story (1995)', 0.05587310329152746),
 ('Men in Black (1997)', 0.00420455868358584)]

# 회고
